In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow
import sys
import os
sys.path.append(os.path.abspath('..'))
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer
from lightgbm import LGBMClassifier
from sklearn.pipeline import make_pipeline

from src.utils import *
from src.aeroclub import *
from src.models import *

In [2]:
train = pd.read_parquet('../data/train.parquet')
print("Train shape:", train.shape)
test = pd.read_parquet('../data/test.parquet')
print("Test shape:", test.shape)
train_df = train.drop(columns='Id')
del train

Train shape: (18145372, 126)
Test shape: (6897776, 125)


In [3]:
train_df = safe_optimize_dtypes(train_df)

In [4]:
cat_cols, num_cols, date_cols, id_cols = get_column_types(train_df)

/Users/jaewoo/Desktop/Kaggle/Aeroclub/src/utils.py:22: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  temp_date = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)
/Users/jaewoo/Desktop/Kaggle/Aeroclub/src/utils.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_date = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)
/Users/jaewoo/Desktop/Kaggle/Aeroclub/src/utils.py:22: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.ht

In [5]:
duration_cols = [col for col in date_cols if 'duration' in col]
true_date_cols = [col for col in date_cols if 'duration' not in col]

In [6]:
train_df, datetime_new_cols = extract_datetime_features(train_df, true_date_cols)

In [8]:
train_df = train_df.drop(columns=true_date_cols)

In [9]:
train_df, duration_new_cols = duration_split(train_df, duration_cols)

In [10]:
train_df = train_df.drop(columns=duration_cols)

In [11]:
cat_cols = cat_cols + datetime_new_cols + duration_new_cols

In [13]:
run_baseline_cv(train_df, cat_cols)

[LightGBM] [Info] Number of positive: 84431, number of negative: 422156
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027540 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8449
[LightGBM] [Info] Number of data points in the train set: 506587, number of used features: 112
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166666 -> initscore=-1.609440
[LightGBM] [Info] Start training from score -1.609440
Fold 1:  0.40175
[LightGBM] [Info] Number of positive: 84431, number of negative: 422156
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028069 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8437
[LightGBM] [Info] Number of data points in the train set: 506587, number of used features:

(np.float64(0.40370002197033444),
 np.float64(0.0010215427542830335),
 [0.4017503366031929,
  0.4043529616792208,
  0.40402299963216076,
  0.40464062288160213,
  0.40373318905549543])